# store

> Store text features to avoid recomputing them.

In [ ]:
#| default_exp store

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import pickle
import os
import hashlib

In [ ]:
#| export
class TextFeatureStore:
	""" A class to store features extracted for a text classification pipeline and cache them to disk to avoid recomputing them. """
	def __init__(self, 
			  	 path:str = None # where to save a pickle file to persist the feature store between runs
				 ):
		self.texts = {}
		self.path = path

		if self.path is not None:
			# self.path shouuld be a file - make dirs if it doesn't exist
			if not os.path.exists(self.path):
				os.makedirs(os.path.dirname(self.path), exist_ok=True)

	def empty(self):
		"""Initialize the feature store."""
		self.texts = {}

	def save(self):
		"""Save the feature store to disk."""
		if self.path is not None:
			with open(self.path, 'wb') as f:
				pickle.dump(self.texts, f)

	def load(self):
		"""Load the feature store to disk."""
		if self.path is not None and os.path.exists(self.path):
			with open(self.path, 'rb') as f:
				self.texts = pickle.load(f)
		return self

	def update(self, 
			   text:str, # the text to update
			   tokens:list, # the tokens to update
			   pos:list, # the part of speech tags to update
			   textstats:list, # the text statistics to update
			   ):
		"""Update the feature store tokens, parts of speech tags and text statistics for a text."""
		hash = hashlib.md5(text.encode()).hexdigest()
		if hash not in self.texts:
			self.texts[hash] = {}
		self.texts[hash]['tokens'] = tokens
		self.texts[hash]['pos'] = pos
		self.texts[hash]['textstats'] = textstats

	def update_embeddings(self, 
					   	  texts:str, # the texts to update 
						  embeddings:list # the embeddings to update
						  ):
		"""Update the feature store with embeddings for a list of texts."""

		for i, text in enumerate(texts):
			hash = hashlib.md5(text.encode()).hexdigest()
			if hash not in self.texts:
				self.texts[hash] = {}
			self.texts[hash]['embeddings'] = embeddings[i]
		
	def get(self, 
		 	text:str, # the text to get features for
			type:str = None # the type of features to get - 'tokens', 'pos', 'textstats', 'embeddings'
			) -> dict|list: # the features for the text
		""" Get features for a text."""

		hash = hashlib.md5(text.encode()).hexdigest()
		if type is None:
			return self.texts.get(hash, None)
		elif type in ['tokens', 'pos', 'textstats', 'embeddings']:
			return self.texts.get(hash, {}).get(type, None)
		else:
			raise ValueError(f"Type {type} not in ['tokens', 'pos', 'textstats', 'embeddings']")
		
	def get_tokens_from_texts(self, 
						      texts:list, # the texts to get tokens for
							  lowercase:bool = False # whether to lowercase the tokens
							  ) -> list: # the tokens for the texts
		""" Get tokens for a list of texts. """
		tokens = []
		for text in texts:
			tokens.append(self.get(text, 'tokens'))
		if lowercase == True:
			tokens = [[token.lower() for token in text] for text in tokens]
		return tokens
	
	def get_textstats_from_texts(self, 
							  	 texts:list, # the texts to get text statistics for
								 columns_out = ['monosyll_count', 'polysyll_count', 'token_count', 'sentence_count', 'unique_tokens_count', 'average_sentence_length'], # the text statistics to get
								 columns_in = ['monosyll_count', 'polysyll_count', 'token_count', 'sentence_count', 'unique_tokens_count', 'average_sentence_length'] # the column name order - leaving for future use, but probably not needed
								 ) -> list: # the text statistics for the texts
		""" Get text statistics for a list of texts. """
		try:
			columns = [columns_in.index(col) for col in columns_out]
		except ValueError:
			raise ValueError(f"Columns {columns_out} not in {columns_in}")
		textstats = []
		for text in texts:
			textstat = self.get(text, 'textstats')
			textstats.append([textstat[i] for i in columns])
		return textstats
	
	def get_pos_from_texts(self, 
						   texts:list, # the texts to get part of speech tags for
						   ) -> list: # the part of speech tags for the texts
		""" Get parts of speech for a list of texts. """
		pos = []
		for text in texts:
			pos.append(self.get(text, 'pos'))
		return pos

	def get_embeddings_from_texts(self, 
							      texts:str # the texts to get embeddings for
								  ) -> list: # the embeddings for the texts
		""" Get embeddings for multiple texts. """
		embeddings = []
		for text in texts:
			embeddings.append(self.get(text, 'embeddings'))
		return embeddings

TODO: add an example.

In [ ]:
#| hide
text_feature_store = TextFeatureStore(path='pipeline/test_feature_store')
text_feature_store.empty()
text_feature_store.update('Hello, world!', ['Hello', 'world'], ['NNP', 'NN'], [1, 1])

assert text_feature_store.get('Hello, world!') == {'tokens': ['Hello', 'world'], 'pos': ['NNP', 'NN'], 'textstats': [1, 1]}
assert text_feature_store.get('Hello, world!', 'tokens') == ['Hello', 'world']
assert text_feature_store.get('Hello, world!', 'pos') == ['NNP', 'NN']
assert text_feature_store.get('Hello, world!', 'textstats') == [1, 1]

text_feature_store.update_embeddings(['Hello, world!'], [[1, 2, 3]])
assert text_feature_store.get('Hello, world!', 'embeddings') == [1, 2, 3]

text_feature_store.empty()
assert text_feature_store.get('Hello, world!', 'tokens') is None
assert text_feature_store.get('Hello, world!', 'pos') is None
assert text_feature_store.get('Hello, world!', 'textstats') is None
assert text_feature_store.get_embeddings_from_texts(['Hello, world!']) == [None]

text_feature_store.update('Hello, world!', ['Hello', 'world'], ['NNP', 'NN'], [1, 1])
text_feature_store.update_embeddings(['Hello, world!'], [[1, 2, 3]])
assert text_feature_store.get_embeddings_from_texts(['Hello, world!']) == [[1, 2, 3]]
assert text_feature_store.get_embeddings_from_texts(['Hello, world!', 'Hi, world!']) == [[1, 2, 3], None]

text_feature_store.save()
assert os.path.exists('pipeline/test_feature_store')

text_feature_store.empty()
assert text_feature_store.get('Hello, world!', 'tokens') is None

text_feature_store.load()
assert text_feature_store.get('Hello, world!') == {'tokens': ['Hello', 'world'], 'pos': ['NNP', 'NN'], 'textstats': [1, 1], 'embeddings': [1, 2, 3]}
text_feature_store.empty()
os.remove('pipeline/test_feature_store')
os.rmdir('pipeline')

text_feature_store = TextFeatureStore()
text_feature_store.update('Hello, world!', ['Hello', 'world'], ['NNP', 'NN'], [1, 1])
assert text_feature_store.get('Hello, world!') == {'tokens': ['Hello', 'world'], 'pos': ['NNP', 'NN'], 'textstats': [1, 1]}
text_feature_store.save()
text_feature_store = text_feature_store.load()
assert text_feature_store.get('Hello, world!') == {'tokens': ['Hello', 'world'], 'pos': ['NNP', 'NN'], 'textstats': [1, 1]}

text_feature_store.update('Hello, world!', ['Hello', 'world'], ['NNP', 'NN'], [1, 1, 2, 1, 2, 1])
assert text_feature_store.get_textstats_from_texts(['Hello, world!']) == [[1, 1, 2, 1, 2, 1]]
assert text_feature_store.get_textstats_from_texts(['Hello, world!'], columns_out=['monosyll_count', 'polysyll_count']) == [[1, 1]]

# this should raise a value error
try:
    textstats = text_feature_store.get_textstats_from_texts(['Hello, world!'], columns_out=['dr_dre'])
except ValueError as e:
    assert True
else:
    assert False, "Expected ValueError not raised"

del text_feature_store

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()